In [1]:
!pip install torch torchvision torchaudio einops pillow tqdm accelerate numpy matplotlib

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
batch_size = 16
epochs = 200  # Increase epochs as needed for better results
image_size = 64  # Resize the images to 64x64 (you can increase to 128 or 256)
learning_rate = 1e-4


In [4]:
# Paths to your dataset
tumor_dir = "data/generation_data/Training/tumor"  # Update with your path
no_tumor_dir = "data/generation_data/Training/notumor"  # Update with your path


In [5]:
# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Grayscale(num_output_channels=1),  # Ensure grayscale images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
])

# Custom Dataset class to load images
class TumorDataset(Dataset):
    def __init__(self, tumor_dir, no_tumor_dir, transform=None):
        self.tumor_images = [os.path.join(tumor_dir, img) for img in os.listdir(tumor_dir)]
        self.no_tumor_images = [os.path.join(no_tumor_dir, img) for img in os.listdir(no_tumor_dir)]
        self.images = self.tumor_images + self.no_tumor_images
        self.labels = [1] * len(self.tumor_images) + [0] * len(self.no_tumor_images)  # 1: Tumor, 0: No Tumor
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = self.transform(img)  # Apply transformations
        label = self.labels[idx]
        return img, label

# Load dataset
dataset = TumorDataset(tumor_dir, no_tumor_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# DDPM model (U-Net architecture for diffusion)
class UNetDDPM(nn.Module):
    def __init__(self):
        super(UNetDDPM, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.middle = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        )
    
    def forward(self, x):
        enc = self.encoder(x)
        middle = self.middle(enc)
        dec = self.decoder(middle)
        return dec

# Instantiate and move model to GPU/CPU
model = UNetDDPM().to(device)


In [ ]:
# Loss function (MSE for DDPM)
loss_fn = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Training loop
def train(model, dataloader, epochs):
    model.train()
    for epoch in range(epochs):
        loop = tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}")
        for batch_idx, (imgs, _) in enumerate(loop):
            imgs = imgs.to(device)
            optimizer.zero_grad()

            # Forward pass
            output = model(imgs)
            loss = loss_fn(output, imgs)  # Compare output with original image (DDPM)
            loss.backward()
            optimizer.step()

            loop.set_postfix(loss=loss.item())

# Start training
train(model, dataloader, epochs)


In [ ]:
# Saving the model
torch.save(model.state_dict(), "unet_ddpm.pth")


In [ ]:
# Function to generate synthetic MRI images
def generate_images(model, num_images=5):
    model.eval()
    with torch.no_grad():
        z = torch.randn(num_images, 1, image_size, image_size).to(device)  # Random noise
        generated_images = model(z)  # Generate images
        generated_images = generated_images.cpu().numpy()

        for i, img in enumerate(generated_images):
            plt.subplot(1, num_images, i + 1)
            plt.imshow(img[0], cmap='gray')
            plt.axis('off')
        plt.show()

# Generate 5 synthetic images after training
generate_images(model, num_images=5)
